In [0]:
import tensorflow as tf
import os

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone dev https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 524 (delta 60), reused 74 (delta 29), pack-reused 413
Receiving objects: 100% (524/524), 271.50 KiB | 5.03 MiB/s, done.
Resolving deltas: 100% (276/276), done.


In [0]:
BUCKET = 'gs://gs_colab' #@param {type:"string"}
PROJECT = 'dvc' #@param {type:"string"}
MODEL = "InceptionResNetV2" #@param ["InceptionResNetV2", "ResNet50", "ResNet50V2", "InceptionV3", "MobileNetV2", "Xception"]

BATCH_SIZE = 128 #@param {type: "integer"}
EPOCHS = 1 #@param {type:"integer"}

In [0]:
fw.colab_tpu.setup_gcs()

In [0]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)


In [0]:
base_model_func, weight_dir, weight_vars, img_size, normalizer = fw.keras_models.get_model(MODEL, BUCKET)

In [0]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.DVC, os.path.join('.', PROJECT))
train_dir_local = os.path.join(data_dir_local, 'dogscats/train')
valid_dir_local = os.path.join(data_dir_local, 'dogscats/valid')

In [0]:
train_fn = os.path.join(data_dir, 'train.tfrec')
valid_fn = os.path.join(data_dir, 'valid.tfrec')
paths_train, y_train, labels_train = fw.io.data_dir_tfrecord(train_dir_local, train_fn)
paths_valid, y_valid, labels_valid = fw.io.data_dir_tfrecord(valid_dir_local, valid_fn)
n_train, n_valid = len(y_train), len(y_valid)
n_classes = len(labels_train)

In [0]:
def build_nn(c=256):
  base_model = base_model_func()
  base_model.trainable = False
  model = fw.Sequential()
  model.add(base_model)
  model.add(tf.keras.layers.Flatten())
  model.add(fw.layers.DenseBlk(c))
  model.add(tf.keras.layers.Dense(n_classes, use_bias=False))
  return model

In [0]:
total_steps = n_train // BATCH_SIZE * EPOCHS

In [0]:
opt_func = fw.train.adam_sgdr_one_cycle(total_steps, lr=0.01)
model_func = fw.tpuest.get_clf_model_func(build_nn, opt_func)

In [0]:
parser_train = fw.io.get_tfexample_image_parser(img_size, img_size, training=True, normalizer=normalizer)
parser_eval = fw.io.get_tfexample_image_parser(img_size, img_size, training=False, normalizer=normalizer)

train_input_func = lambda params: fw.io.tfrecord_ds(train_fn, parser_train,
  params['batch_size'], training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(valid_fn, parser_eval, 
  params['batch_size'], training=False)

In [13]:
est = fw.tpuest.get_tpu_estimator(n_train, n_valid, model_func, work_dir, 
                             weight_dir, weight_vars, BATCH_SIZE)

est.train(train_input_func, steps=total_steps)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/dvc/2019-04-12-03:37:15', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.65.51.74:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbad0d2c278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.65.51.74:8470', '_evaluation_master': 'grpc://10.65.51.74:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=179, num_shards=None, num_co

In [14]:
result = est.evaluate(valid_input_func, steps=1)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-12T03:40:38Z
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gs://gs_colab/work/dvc/2019-04-12-03:37:15/model.ckpt-179
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 7 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:

In [15]:
print(f'Test results: accuracy={result["accuracy"] * 100: .2f}%, loss={result["loss"]: .2f}.')

Test results: accuracy= 99.35%, loss= 0.04.


In [0]:
fw.io.create_clean_dir(work_dir)